### 1 - Imports

In [834]:
import os
import tiktoken
import difflib
import pandas as pd
import io
import sys
from dotenv import load_dotenv
from firecrawl import FirecrawlApp
from langchain_openai import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import requests
import urllib.parse


### 2 - OpenAI Azure models deployement

In [835]:
load_dotenv()

True

In [836]:

llm_4o = AzureChatOpenAI(
    azure_endpoint=os.getenv('Azure_OpenAI_OB_Endpoint'), 
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4o",
    openai_api_key=os.getenv('Azure_OpenAI_OB_Key'), 
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt-4o-deploy")
    #streaming=True

In [837]:
llm_4omini = AzureChatOpenAI(
    azure_endpoint=os.getenv('Azure_OpenAI_OB_Endpoint_4mini'), 
    openai_api_version="2024-10-21",   
    model_name="gpt-4o-mini",
    openai_api_key=os.getenv('Azure_OpenAI_OB_Key_4mini'), 
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt4o-mini")
    #streaming=True

In [838]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=os.getenv('Azure_OpenAI_emb_3_large_azure_endpoint'),
    openai_api_key=os.getenv('Azure_OpenAI_emb_3_large_api_key'),
    openai_api_version="2023-05-15",
    chunk_size=1000
)


In [839]:
prompt = "Fais une blague sur les actuaires"

for chunk in llm_4o.stream(prompt):
    print(chunk.content, end="", flush=True)

Pourquoi les actuaires ne racontent-ils jamais de blagues lors des fêtes ?

Parce que les probabilités que quelqu'un rie sont trop faibles !

In [840]:
for chunk in llm_4omini.stream(prompt):
    print(chunk.content, end="", flush=True)

Pourquoi les actuaires ne racontent-ils jamais de blagues ?

Parce qu'ils préfèrent les statistiques à l'humour, et que les rires ne sont pas toujours prévisibles !

In [841]:
sentence1 = "test test test "
emb1 = embeddings.embed_query(sentence1)
print(emb1)

[-0.015937637261025525, 0.01671048597292706, -0.006908508829043585, 0.04772809064974347, 0.024731142017042972, 0.010593675824367968, -0.01671048597292706, 0.0607345615214083, -0.02684233849833496, 0.015145940443186146, -0.0013006470173172909, 0.032893172825531114, -0.021507801114646327, -0.007940544608960876, 0.004281296320438067, 0.026012939880684538, -0.005183738100686199, 0.018180783522398266, -0.033873371191845254, -0.002001629641919673, 0.0037016602521731975, -0.01896305535594616, -0.007082871038436004, 0.01292164508171893, -0.037699911088896754, 0.016079012025397757, 0.027841384970586948, -0.00581521158255422, 0.0010550084806358584, -0.03172447849867757, 0.007115858483456191, 0.03445772021791715, -0.0029547307127345207, 0.004639445723514387, 0.018774557532761638, 0.0009672383726290914, 0.01829388333389605, 0.024259894665113985, 0.007605957200951982, 0.0026319252335818984, -0.004677145660680316, -0.022733047209893883, -0.0151176654903117, -0.00016508438861708629, -0.02203559837232

### 3 -  Data pipeline 

#### 3.0 -  Token counting 

In [842]:
def count_tokens(text: str, model: str = "gpt-4o") -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

#### 3.1 -  Data importing pipeline

##### 3.1.1 -  Firecrawl (URL -> Markdown , HTML)

In [843]:
url_aml_5="http://publications.europa.eu/resource/celex/32015L0849"
url_crr="http://publications.europa.eu/resource/celex/32013R0575"

In [844]:
app = FirecrawlApp(api_key=os.getenv('api_key_firecraw_2'))

In [845]:
scrape_result_crr = app.scrape_url(
    url_crr,
    params={
        'formats': ['markdown', 'html'],
        'location': {
            'country': 'FR'
        }
    }
)

In [846]:
scrape_result_AML_5 = app.scrape_url(
    url_aml_5,
    params={
        'formats': ['markdown', 'html'],
        'location': {
            'country': 'FR'
        }
    }
)

In [891]:
directive_UE_AML5_mrkd =  scrape_result_AML_5["markdown"] 
count_tokens(directive_UE_AML5_mrkd)

50960

In [892]:
Reglement_UE_CRR_mrkd =  scrape_result_crr["markdown"]
count_tokens(directive_UE_AML5_mrkd)

50960

##### 3.1.2 -  URL to PDF

In [849]:
def url_to_pdf(url, save_path):
    response = requests.get(url)
    with open(save_path, "wb") as f:
        f.write(response.content)

In [850]:
url_to_pdf("https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32018L0843", 
                "/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_input/AML5.pdf")

##### 3.1.3 -  PDF loader

In [851]:
def pdf_loader(path):
    loader = PyPDFLoader(file_path=path, mode='single')
    doc= loader.load()
    return doc

In [896]:
document_aml_pdf= pdf_loader("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_input/AML5.pdf")
document_aml_pdf_str= str(document_aml_pdf)

In [895]:
document_crr_pdf= pdf_loader("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_input/CRR.pdf")
document_crr_pdf_str= str(document_crr_pdf)

#### 3.2 -  Token optimization pipelines

##### 3.2.1 - Directive data pipeline cleaning

Le contenu de la directive commence par : "ONT ADOPTÉ LA PRÉSENTE DIRECTIVE:"

Et se termine avant "Par le Parlement européen\n\nLe président\"

Solution : On supprime donc le contenu avant cette phrase avec les deux fonctions : supr_avant_directive + supr_apres_directive

In [893]:
def supr_avant_directive(texte):
    start_phrase = "ONT ADOPTÉ LA PRÉSENTE DIRECTIVE:\n\nCHAPITRE I"
    index_debut = texte.find(start_phrase)
    if index_debut != -1:
        return texte[index_debut:]
    else:
        return "Le texte ne contient pas la phrase spécifiée."

In [855]:
def supr_apres_directive(texte):
    end_phrase = "Par le Parlement européen\n\nLe président\n"
    index_fin = texte.find(end_phrase)
    if index_fin != -1:
        return texte[:index_fin + len(end_phrase)]
    else:
        return texte  

In [897]:
document_aml_pdf_step_1=supr_avant_directive(directive_UE_AML5_mrkd)
count_tokens(document_aml_pdf_step_1) # gain de 13K token

37286

In [899]:
document_aml_pdf_step_2=supr_apres_directive(document_aml_pdf_step_1)
count_tokens(document_aml_pdf_step_2) # gain de 8K/9k token

29819

Optionnel : Suppression des caractères markdown

In [900]:
import re

def nettoyer_markdown(texte):
    # Supprimer les tableaux Markdown
    texte = re.sub(r'\|.*?\|', '', texte)
    texte = re.sub(r'\|\s*---\s*\|', '', texte)
    
    # Supprimer les liens Markdown
    texte = re.sub(r'\[.*?\]\(.*?\)', '', texte)
    
    # Supprimer les annotations de type [texte][numéro]
    texte = re.sub(r'\[.*?\]\[\d+\]', '', texte)
    
    # Supprimer les espaces multiples et les lignes vides consécutives
    texte = re.sub(r'\n\s*\n', '\n', texte)
    texte = re.sub(r'[ ]{2,}', ' ', texte)
    
    # Supprimer les caractères spéciaux isolés comme '|'
    texte = re.sub(r'\|', '', texte)
    
    return texte.strip()

In [901]:
document_aml_pdf_step_3 = nettoyer_markdown(document_aml_pdf_step_2)
count_tokens(document_aml_pdf_step_3) # gain de plus de 3k token

26144

- Step 1: gain de 13k token
- Step 2: gain de 8k/9K token
- Step 3: gain de 3k token

In [902]:
document_aml_pdf_step_3

"ONT ADOPTÉ LA PRÉSENTE DIRECTIVE:\nCHAPITRE I\nDISPOSITIONS GÉNÉRALES\nSECTION\xa01\nObjet, champ d'application et définitions\nArticle premier\n1.\xa0\xa0\xa0La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme.\n2.\xa0\xa0\xa0Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits.\n3.\xa0\xa0\xa0Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les\xa0agissements ci-après énumérés, commis intentionnellement:\n la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux\xa0conséquences juridiques des actes qu'elle a commis; \n le fait de dissimuler ou 

In [903]:
def weird_carac_remove(variable):
    variable_1 = variable.replace("\xa0", " ")
    variable_2 = variable_1.replace("\n", " ")
    return variable_2

In [904]:
document_aml_pdf_step_4 = weird_carac_remove(document_aml_pdf_step_3)
count_tokens(document_aml_pdf_step_4)

25891

#### 3.2.2 - Reglement data pipeline cleaning

In [905]:
def supr_avant_reglement(texte):
    start_phrase = "ONT ADOPTÉ LE PRÉSENT RÈGLEMENT"
    index_debut = texte.find(start_phrase)
    if index_debut != -1:
        return texte[index_debut:]
    else:
        return "Le texte ne contient pas la phrase spécifiée."

In [906]:
count_tokens(directive_UE_CRR)

408669

In [907]:
directive_UE_CRR_step1 = supr_apres_directive(directive_UE_CRR)
count_tokens(directive_UE_CRR_step1)

373977

In [908]:
def supr_apres_reglement(texte):
    end_phrase = "Par le Parlement européen"
    index_fin = texte.find(end_phrase)
    if index_fin != -1:
        return texte[:index_fin + len(end_phrase)]
    else:
        return texte  

In [909]:
directive_UE_CRR_step2= supr_apres_reglement(directive_UE_CRR_step1)

In [910]:
count_tokens(directive_UE_CRR_step2)

373973

In [911]:
directive_UE_CRR_step3=nettoyer_markdown(directive_UE_CRR_step2)

In [912]:
directive_UE_CRR_step4=weird_carac_remove(directive_UE_CRR_step3)

In [913]:
directive_UE_CRR_step4
count_tokens(directive_UE_CRR_step4)

315607

#### 3.2.3 Snowball stemmer

In [914]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [915]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /Users/oussa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/oussa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [916]:
def preprocess_text_snowball(texte, langue='french'):
    """
    Cette fonction prend en entrée un texte, supprime les stopwords et applique le SnowballStemmer,
    puis retourne le texte prétraité sous forme de chaîne de caractères.
    
    :param texte: Le texte à traiter (chaîne de caractères).
    :param langue: La langue du texte et du stemmer (par défaut 'french').
    :return: Une chaîne de caractères du texte prétraité.
    """
    # Initialisation du stemmer pour la langue spécifiée
    stemmer = SnowballStemmer(langue)
    
    # Chargement des stopwords pour la langue spécifiée
    stop_words = set(stopwords.words(langue))
    
    # Tokenisation du texte
    mots = word_tokenize(texte, language=langue)
    
    # Filtrage des stopwords et racinisation des mots restants
    mots_filtrés_et_racinisés = [
        stemmer.stem(mot) for mot in mots
        if mot.isalnum() and mot.lower() not in stop_words
    ]
    
    # Conversion de la liste de mots en une chaîne de caractères
    texte_pretraite = ' '.join(mots_filtrés_et_racinisés)
    
    return texte_pretraite

In [ ]:
directive_UE_CRR_stemed = preprocess_text_snowball(directive_UE_CRR_step4)
print("Nombre de token :",f"{count_tokens(directive_UE_CRR_stemed)}")

Nombre de token : 161281


#### 3.2.4 Semantic compression - To do ?

Segmentant le texte en blocs thématiques avant de les résumer. Permet de traiter des textes 6 à 8 fois plus longs. Préserve mieux la structure thématique et la cohérence globale du texte.

Papier de recherche: https://arxiv.org/abs/2304.12512

#### 3.2.5 Selective Context - To do ?

Pour l’identification et la suppression des redondances. Fournit des mesures plus précises de réduction de la mémoire et du temps d’inférence. Réduit de 50% le coût du contexte. Peut parfois conduire à la perte d’informations importantes. Négliger les interdépendances entre les contenus compressés.

Papier de recherche: https://arxiv.org/abs/2310.06201

### 4 - Option 1: Long Context Question Answering

Pour le versionning de prompts dans Python, plusieurs outils possibles :
- PromptLayer
- LangSmith , 
- Agenta, 
- Mirascope et Lilypad
- prompt flow

In [918]:
fr_old_version= "Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7 , L. 632-13 et L. 632-16 du présent code sont tenues, nonobstant toutes dispositions contraires, de transmettre à des entreprises du même groupe ayant leur siège social dans l'un de ces Etats : 1° Les renseignements relatifs à leur situation financière nécessaires à l'organisation de la surveillance sur base consolidée et de la surveillance complémentaire de ces entités réglementées ou sociétés de financement ; 2° Les informations nécessaires à l'organisation de la lutte contre le blanchiment des capitaux et contre le financement du terrorisme ; 3° Les informations nécessaires à l'organisation de la détection des abus de marché mentionnés à l'article 16 du règlement (UE) n° 596/2014 du Parlement européen et du Conseil du 16 avril 2014 sur les abus de marché (règlement relatif aux abus de marché) et abrogeant la directive 2003/6/CE du Parlement européen et du Conseil et les directives 2003/124/CE, 2003/125/CE et 2004/72/CE de la Commission ; 4° Les informations nécessaires à la gestion des conflits d'intérêts au sens du 3° du I et 3° du II de l'article L. 533-10 . Ces dernières informations ne peuvent être communiquées à des personnes extérieures au groupe, à l'exception des autorités compétentes des Etats visés au premier alinéa. Cette exception ne s'étend pas aux autorités des Etats ou territoires dont la législation est reconnue insuffisante ou dont les pratiques sont considérées comme faisant obstacle à la lutte contre le blanchiment des capitaux ou le financement du terrorisme par l'instance internationale de concertation et de coordination en matière de lutte contre le blanchiment d'argent dont la liste est mise à jour par arrêté du ministre chargé de l'économie. Les personnes recevant ces informations sont tenues au secret professionnel dans les conditions et sous les peines mentionnées à l'article L. 511-33 , pour tous renseignements ou documents qu'elles seraient ainsi amenées à recevoir ou à détenir. Les dispositions du présent article ne font pas obstacle à l'application de la loi n° 78-17 du 6 janvier 1978 relative à l'informatique, aux fichiers et aux libertés."

In [919]:
fr_new_version= "Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, d'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7 , L. 632-13 et L. 632-16 du présent code sont tenues, nonobstant toutes dispositions contraires, de transmettre à des entreprises du même groupe ayant leur siège social dans l'un de ces Etats : 1° Les renseignements relatifs à leur situation financière nécessaires à l'organisation de la surveillance sur base consolidée et de la surveillance complémentaire de ces entités réglementées ou sociétés de financement ; 2° Les informations nécessaires à l'organisation de la lutte contre le blanchiment des capitaux et contre le financement du terrorisme ; 3° Les informations nécessaires à l'organisation de la détection des abus de marché mentionnés à l'article 16 du règlement (UE) n° 596/2014 du Parlement européen et du Conseil du 16 avril 2014 sur les abus de marché (règlement relatif aux abus de marché) et abrogeant la directive 2003/6/CE du Parlement européen et du Conseil et les directives 2003/124/CE, 2003/125/CE et 2004/72/CE de la Commission ; 4° Les informations nécessaires à la gestion des conflits d'intérêts au sens du 3° du I et 3° du II de l'article L. 533-10 . Ces dernières informations ne peuvent être communiquées à des personnes extérieures au groupe, à l'exception des autorités compétentes des Etats visés au premier alinéa. Cette exception ne s'étend pas aux autorités des Etats ou territoires dont la législation est reconnue insuffisante ou dont les pratiques sont considérées comme faisant obstacle à la lutte contre le blanchiment des capitaux ou le financement du terrorisme par l'instance internationale de concertation et de coordination en matière de lutte contre le blanchiment d'argent dont la liste est mise à jour par arrêté du ministre chargé de l'économie. Les personnes recevant ces informations sont tenues au secret professionnel dans les conditions et sous les peines mentionnées à l'article L. 511-33 , pour tous renseignements ou documents qu'elles seraient ainsi amenées à recevoir ou à détenir. Les dispositions du présent article ne font pas obstacle à l'application de la loi n° 78-17 du 6 janvier 1978 relative à l'informatique, aux fichiers et aux libertés."

In [920]:
output_buffer_4o = io.StringIO()

#### 4.1 - Version 1 - 3p 

Mise en place d'une fonction de récupération des textes européens avec un prompt a 3 paramètres:
- Le réglement UE 
- l'ancienne version du text
- la nouvelle version du text

In [921]:
# LLM produit : analyse du changement avant et après et recherche de l article UE a l origine du changement.
prompt_LCQA_3p = f""" 
        Tu es un juriste expert avec au moins 10 ans d expérience.
        Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
        Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
        
        Ta tâche :
        -   Analyser les changements réglementaires dans la loi francaise avec pour objectif de trouver la ou les parties
        de la directive UE (article ou articles) a l origine de ce(s) changement(s).
        
        Régles : 
        - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
        - Tu feras ton analyse de la facon la plus conconcieuse et précise 
        - Tu ne pas pas reciter le contenu de la loi francaise avant et après modification
        
        Voici le text européen {document_aml_pdf_step_4}
        Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
        Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
"""

In [922]:
for chunk in llm_4o.stream(prompt_LCQA_3p):
    print(chunk.content, end="", flush=True)

Pour analyser les changements réglementaires dans la loi française et identifier les parties de la directive européenne à l'origine de ces changements, nous devons examiner les modifications apportées au texte de loi français avant et après l'entrée en vigueur de la directive européenne.

### Analyse des Changements

1. **Ajout de la référence à l'article L. 511-20 du Code monétaire et financier**:
   - **Avant**: Le texte ne mentionnait pas l'article L. 511-20.
   - **Après**: Le texte inclut "d'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2".

2. **Inclusion des entités mentionnées au 1° bis ou 1° ter de l'article L. 561-2**:
   - **Avant**: Le texte ne faisait pas référence aux entités mentionnées au 1° bis ou 1° ter de l'article L. 561-2.
   - **Après**: Le texte inclut explicitement ces entités.

### Correspondance avec la Directive Européenne

Les modifications apportées à la loi française semble

In [923]:
def get_eu_data_3p(directive_UE_CRR_step4, fr_old_version, fr_new_version):
    prompt_LCQA_3p = f""" 
        Tu es un juriste expert avec au moins 10 ans d expérience.
        Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
        Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
        
        Ta tâche :
        -   Analyser les changements réglementaires dans la loi francaise avec pour objectif de trouver la ou les parties
        de la directive UE (article ou articles) a l origine de ce(s) changement(s).
        
        Régles : 
        - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
        - Tu feras ton analyse de la facon la plus conconcieuse et précise 
        - Tu ne pas pas reciter le contenu de la loi francaise avant et après modification
        
        Voici le text européen {document_aml_pdf_step_4}
        Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
        Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
"""
    for chunk in llm_4o.stream(prompt_LCQA_3p):
        print(chunk.content, end="", flush=True)

In [924]:
get_eu_data_3p(directive_UE_CRR_step4, fr_old_version, fr_new_version )

Pour analyser les changements réglementaires dans la loi française et identifier les parties de la directive européenne à l'origine de ces changements, nous devons comparer les versions avant et après l'entrée en vigueur du texte européen et relier ces modifications aux articles pertinents de la directive.

### Analyse des Changements

**Avant l'entrée en vigueur du texte européen:**
- Les entreprises établies en France et faisant partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement, ou d'un groupe au sens de l'article L. 356-1 du code des assurances, ou d'un groupe mixte ou d'un conglomérat financier, doivent transmettre certaines informations à des entreprises du même groupe ayant leur siège social dans l'UE ou l'EEE, ou dans un État avec des accords spécifiques.
- Les informations à transmettre incluent celles nécessaires à la lutte contre le blanchiment des capitaux et le financement du terrorisme.

**Après l'entrée en vigueur du texte européen

#### 4.2 - Version 2 - 4p 

Rajout d'une fonction de dectection de changement entre les deux textes pour orienter l'analyse LLM

In [925]:
def comparer_phrases(phrase1, phrase2):
    mots1 = phrase1.split()
    mots2 = phrase2.split()
    sequence = difflib.SequenceMatcher(None, mots1, mots2)
    differences = []
    for tag, i1, i2, j1, j2 in sequence.get_opcodes():
        if tag == 'replace':
            differences.append(f"Remplacer '{' '.join(mots1[i1:i2])}' par '{' '.join(mots2[j1:j2])}'")
        elif tag == 'delete':
            differences.append(f"Supprimer '{' '.join(mots1[i1:i2])}'")
        elif tag == 'insert':
            differences.append(f"Insérer '{' '.join(mots2[j1:j2])}'")
    return differences

In [926]:
changement_derter= comparer_phrases(fr_old_version, fr_new_version)
print(changement_derter)

["Insérer 'd'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2,'"]


In [927]:
prompt_LCQA_4p = f""" 
        Tu es un juriste expert avec au moins 10 ans d expérience.
        Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
        Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
        
        Ta tâche :
        -   Analyser les changements réglementaires dans la loi francaise et de trouver la ou les parties
        de la directive UE (article ou articles) a l origine de ce changement réglementaire.
        
        Régles : 
        - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
        - Tu feras ton analyse de la facon la plus conconcieuse et précise 
        
        Voici le text européen {document_aml_pdf_step_4}
        Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
        Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
        Voici une indication des changements entre les deux version du texte francais sur la quelle tu t appuieras pour
        trouver la partie du texte europeen a l origine de ce changement : {changement_derter}

"""

In [928]:
for chunk in llm_4o.stream(prompt_LCQA_4p):
    print(chunk.content, end="", flush=True)

Pour analyser les changements réglementaires dans la loi française et identifier la partie de la directive européenne à l'origine de ce changement, nous devons examiner la modification spécifique apportée au texte français et la comparer avec les dispositions de la directive européenne.

### Changement dans le texte français

**Avant l'entrée en vigueur du texte européen :**
Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7, L. 632-13 et L. 632-16 du présent code 

In [ ]:
def get_eu_data_4p(document_aml_pdf_step_4, fr_old_version, fr_new_version):
    changement_derter= comparer_phrases(fr_old_version, fr_new_version)
    print(changement_derter)
    prompt_LCQA_4p = f""" 
            Tu es un juriste expert avec au moins 10 ans d expérience.
            Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
            Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
            
            
            Ta tâche :
            -   Analyser les changements réglementaires dans la loi francaise et de trouver la ou les parties
            de la directive UE (article ou articles) a l origine de ce changement réglementaire.
            
            Régles : 
            - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
            - Tu feras ton analyse de la facon la plus conconcieuse et précise 
            
            Voici le text européen {document_aml_pdf_step_4}
            Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
            Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
            Voici une indication des changements entre les deux version du texte francais sur la quelle tu t appuieras pour
            trouver la partie du texte europeen a l origine de ce changement : {changement_derter}

    """
    for chunk in llm_4o.stream(prompt_LCQA_4p):
        print(chunk.content, end="", flush=True)
        
                    
        

In [930]:
count_tokens(document_aml_pdf_step_4)

25891

In [931]:
get_eu_data_4p(document_aml_pdf_step_4, fr_old_version, fr_new_version)
# RUN 1: article 2 + article 45 + article 9 + articles 37 et 41
# RUN 2: article 46 + article 45 + 45 paragraphe 8 + article 9 + articles 39
# RUN 3: l'article 45 de la directive (UE) 2015/849.

["Insérer 'd'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2,'"]
Pour analyser les changements réglementaires dans la loi française et identifier la ou les parties de la directive européenne à l'origine de ces changements, nous devons examiner les modifications apportées au texte français et les comparer avec les dispositions de la directive européenne.

### Changements dans la loi française

**Avant l'entrée en vigueur du texte européen:**
Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord

#### 4.2 - Testing on some Code monétaire et financier articles 

##### 4.2.1 - Exemple 1 : L561-2 du cmf (prompt v1 - 3p)

In [932]:
fr_old_version="Sont assujettis aux obligations prévues par les dispositions des sections 2 à 7 du présent chapitre : 1° Les organismes, institutions et services régis par les dispositions du titre Ier du présent livre, y compris les succursales des établissements de crédit mentionnés à l'article L. 511-22 et des établissements financiers mentionnés à l'article L. 511-23 ; 1° bis Les établissements de paiement régis par les dispositions du chapitre II du titre II du présent livre y compris les succursales des établissements de paiement mentionnés au II de l'article L. 522-13 ; 1° ter Les établissements de monnaie électronique régis par le chapitre VI du titre II du présent livre y compris les succursales des établissements de monnaie électronique mentionnés à l'article L. 526-24 ; 1° quater Les établissements de crédit, les établissements de paiement, les établissements de monnaie électronique et les prestataires de services sur crypto-actifs ayant leur siège social dans un autre Etat membre de l'Union européenne ou partie à l'Espace économique européen en tant qu'ils exercent leur activité sur le territoire national en ayant recours aux services d'un ou plusieurs agents pour la fourniture de services de paiement en France ou d'une ou plusieurs personnes en vue de distribuer en France de la monnaie électronique et les prestataires de services sur crypto-actifs au sens de l'article L. 525-8 ou d'une ou plusieurs personnes en vue de la fourniture de services sur cryptoactifs ; 2° Les entreprises mentionnées aux articles L. 310-1 et L. 310-2 du code des assurances ; 2° bis Les institutions ou unions régies par le titre III du livre IX du code de la sécurité sociale ; 2° ter Les mutuelles et unions réalisant des opérations mentionnées au 1° du I de l'article L. 111-1 du code de la mutualité ; 2° quater Les fonds de retraite professionnelle supplémentaire mentionnés à l'article L. 381-1 du code des assurances ; 2° quinquies Les mutuelles ou unions de retraite professionnelle supplémentaire mentionnées à l'article L. 214-1 du code de la mutualité ; 2° sexies Les institutions de retraite professionnelle supplémentaire mentionnées à l'article L. 942-1 du code de la sécurité sociale ; 3° Les intermédiaires en opérations de banque et en services de paiement mentionnés à l'article L. 519-1 lorsqu'ils agissent en vertu d'un mandat délivré par un client et qu'ils se voient confier des fonds en tant que mandataire des parties ; 3° bis Les intermédiaires d'assurance définis à l'article L. 511-1 du code des assurances sauf ceux qui agissent sous l'entière responsabilité de l'organisme ou du courtier d'assurance ; 4° Les intermédiaires en financement participatif mentionnés à l'article L. 548-2 ; 5° La Banque de France, l'institut d'émission des départements d'outre-mer mentionné à l'article L. 721-7 du présent code et l'institut d'émission d'outre-mer mentionné à l'article L. 721-18 et L. 721-19 du même code ; 6° Les entreprises d'investissement, y compris les succursales d'entreprises d'investissement mentionnées à l'article L. 532-18-1 ainsi que les succursales d'entreprises d'investissement mentionnées à l'article L. 532-48 les personnes mentionnées à l'article L. 440-2, les entreprises de marché mentionnées à l'article L. 421-2, les dépositaires centraux mentionnés aux 1° et 2° du I de l'article L. 441-1 et gestionnaires de systèmes de règlement et de livraison d'instruments financiers, les conseillers en investissements financiers, les prestataires de services de financement participatif au titre de leurs activités mentionnées à l'article L. 547-4 et les intermédiaires habilités mentionnés à l'article L. 211-4, ainsi que les placements collectifs mentionnés au I de l'article L. 214-1 et les sociétés de gestion de placements collectifs mentionnées à l'article L. 543-1 et les succursales des sociétés de gestion européennes d'OPCVM et de FIA mentionnées aux articles L. 532-20-1 et L. 532-21-3 ; 6° bis Les prestataires de services d'investissement ayant leur siège social dans un autre Etat membre de l'Union européenne en tant qu'ils exercent leur activité sur le territoire national en ayant recours à des agents liés mentionnés à l'article L. 545-1 du code monétaire et financier ; 7° Les changeurs manuels ; 7° bis a) Les prestataires des services mentionnés aux 1° à 4° de l'article L. 54-10-2, et ; b) Les prestataires des services autorisés à fournir des services sur crypto-actifs conformément à l'article 59 du règlement (UE) 2023/1114 du Parlement européen et du Conseil du 31 mai 2023 sur les marchés de crypto-actifs et modifiant les règlements (UE) n° 1093/2010 et (UE) n° 1095/2010 et les directives 2013/36/ UE et (UE) 2019/1937, y compris les succursales établies en France des prestataires ayant leur siège social dans un autre Etat membre de l'Union européenne ou partie à l'Espace économique européen à l'exception de ceux mentionnés au b du 7° quater du présent article ; 7° ter (Abrogé) ; 7° quater a) Les prestataires agréés au titre de l'article L. 54-10-5, à l'exception des prestataires mentionnés au a du 7° bis du présent article ; et b) Les conseillers en investissements financiers, dépositaires centraux et sociétés de gestion de placements collectifs mentionnées à l'article L. 543-1 autorisés à fournir des services sur crypto-actifs conformément à l'article 59 du règlement (UE) 2023/1114 du Parlement européen et du Conseil du 31 mai 2023 sur les marchés de crypto actifs et modifiant les règlements (UE) n° 1093/2010 et (UE) n° 1095/2010 et les directives 2013/36/ UE et (UE) 2019/1937, y compris les succursales établies en France des sociétés de gestion européennes d'OPCVM et de FIA mentionnées aux articles L. 532-20-1 et L. 532-21-3 ayant leur siège social dans un autre Etat membre de l'Union européenne ou partie à l'Espace économique européen ou les prestataires de services autorisés uniquement pour fournir les services de gestion de portefeuille de crypto-actifs ou de conseils en crypto-actifs mentionnés respectivement au i et au h du point 16 du paragraphe 1 de l'article 3 du même règlement ; 8° Les personnes exerçant les activités mentionnées au 1°, mais concernant leur activité de location uniquement en exécution d'un mandat de transaction de biens immeubles dont le loyer mensuel est supérieur ou égal à 10 000 euros, ainsi qu'aux 2°, 4°, 5° et 8° de l'article 1er de la loi n° 70-9 du 2 janvier 1970 réglementant les conditions d'exercice des activités relatives à certaines opérations portant sur les immeubles et les fonds de commerce ; 9° Les opérateurs de jeux ou de paris autorisés sur le fondement de l'article L. 321-1 et L. 321-3 du code de la sécurité intérieure, du V de l'article 34 de la loi n° 2017-257 du 28 février 2017 relative au statut de Paris et à l'aménagement métropolitain et leurs représentants légaux et directeurs responsables ; 9° bis Les opérateurs de jeux ou de paris autorisés sur le fondement de l'article 21 de la loi n° 2010-476 du 12 mai 2010 relative à l'ouverture à la concurrence et à la régulation du secteur des jeux d'argent et de hasard en ligne et leurs représentants légaux, de l'article 5 de la loi du 2 juin 1891, ayant pour objet de réglementer l'autorisation et le fonctionnement des courses de chevaux, de l'article 137 de la loi n° 2019-486 du 22 mai 2019 relative à la croissance et la transformation des entreprises ; 10° Les personnes qui négocient des œuvres d'art et des antiquités ou agissent en qualité d'intermédiaires dans le commerce des œuvres d'art et d'antiquités, y compris lorsque celui-ci est réalisé par des galeries d'art, lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros et les personnes qui entreposent ou négocient des œuvres d'art ou agissent en qualité d'intermédiaires dans le commerce des œuvres d'art quand celui-ci est réalisé dans des ports francs ou zones franches, lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros ; 11° Les personnes acceptant des paiements en espèces ou au moyen de monnaie électronique d'un montant supérieur à un seuil fixé par décret et se livrant au commerce de biens ; 11° bis Les personnes, autres que celles mentionnées aux 1° à 7° quater, se livrant à titre habituel et principal au commerce de métaux précieux ou de pierres précieuses, lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros ; 12° Les experts-comptables, les salariés autorisés à exercer la profession d'expert-comptable en application des articles 83 ter et 83 quater de l'ordonnance n° 45-2138 du 19 septembre 1945 portant institution de l'ordre des experts-comptables et réglementant les titres et la profession d'expert-comptable ; 12° bis Les commissaires aux comptes ; 13° Les avocats au Conseil d'Etat et à la Cour de cassation, les avocats, les notaires, les commissaires de justice, les administrateurs judiciaires et les mandataires judiciaires, dans les conditions prévues à l'article L. 561-3 ; 14° Les opérateurs de ventes volontaires de meubles aux enchères publiques lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros ; 15° Les personnes exerçant l'activité de domiciliation mentionnée aux articles L. 123-11-2 et suivants du code de commerce ; 16° Les personnes exerçant l'activité d'agents sportifs mentionnés à l'article L. 222-7 du code du sport ; 17° Les personnes autorisées au titre du I de l'article L. 621-18-5 ; 18° Les caisses des règlements pécuniaires des avocats créées en application du 9° de l'article 53 de la loi n° 71-1130 du 31 décembre 1971 au titre des fonds, effets ou valeurs déposés par les avocats pour le compte de leurs clients dans le cadre des activités mentionnées au I de l'article L. 561-3 ; 19° Les greffiers des tribunaux de commerce mentionnés à l'article L. 741-1 du code de commerce ; 20° Les gestionnaires de crédits. Les personnes assujetties mentionnées aux 1° à 20° comprennent les personnes physiques et les personnes morales."

In [1075]:
fr_new_version="Sont assujettis aux obligations prévues par les dispositions des sections 2 à 7 du présent chapitre : 1° Les organismes, institutions et services régis par les dispositions du titre Ier du présent livre, y compris les succursales des établissements de crédit mentionnés à l'article L. 511-22 et des établissements financiers mentionnés à l'article L. 511-23 ; 1° bis Les établissements de paiement régis par les dispositions du chapitre II du titre II du présent livre y compris les succursales des établissements de paiement mentionnés au II de l'article L. 522-13 ; 1° ter Les établissements de monnaie électronique régis par le chapitre VI du titre II du présent livre y compris les succursales des établissements de monnaie électronique mentionnés à l'article L. 526-24 ; 1° quater Les établissements de crédit, les établissements de paiement et les établissements de monnaie électronique ayant leur siège social dans un autre Etat membre de l'Union européenne ou partie à l'Espace économique européen en tant qu'ils exercent leur activité sur le territoire national en ayant recours aux services d'un ou plusieurs agents pour la fourniture de services de paiement en France ou d'une ou plusieurs personnes en vue de distribuer en France de la monnaie électronique au sens de l'article L. 525-8 ; 2° Les entreprises mentionnées aux articles L. 310-1 et L. 310-2 du code des assurances ; 2° bis Les institutions ou unions régies par le titre III du livre IX du code de la sécurité sociale ; 2° ter Les mutuelles et unions réalisant des opérations mentionnées au 1° du I de l'article L. 111-1 du code de la mutualité ; 2° quater Les fonds de retraite professionnelle supplémentaire mentionnés à l'article L. 381-1 du code des assurances ; 2° quinquies Les mutuelles ou unions de retraite professionnelle supplémentaire mentionnées à l'article L. 214-1 du code de la mutualité ; 2° sexies Les institutions de retraite professionnelle supplémentaire mentionnées à l'article L. 942-1 du code de la sécurité sociale ; 3° Les intermédiaires en opérations de banque et en services de paiement mentionnés à l'article L. 519-1 lorsqu'ils agissent en vertu d'un mandat délivré par un client et qu'ils se voient confier des fonds en tant que mandataire des parties ; 3° bis Les intermédiaires d'assurance définis à l' article L. 511-1 du code des assurances sauf ceux qui agissent sous l'entière responsabilité de l'organisme ou du courtier d'assurance ; 4° Les intermédiaires en financement participatif mentionnés à l'article L. 548-2 ; 5° La Banque de France, l'institut d'émission des départements d'outre-mer mentionné à l'article L. 711-2 du présent code et l'institut d'émission d'outre-mer mentionné à l'article L. 712-4 du même code ; 6° Les entreprises d'investissement, y compris les succursales d'entreprises d'investissement mentionnées à l'article L. 532-18-1 ainsi que les succursales d'entreprises d'investissement mentionnées à l'article L. 532-48, les personnes mentionnées à l'article L. 440-2 , les entreprises de marché mentionnées à l'article L. 421-2 , les dépositaires centraux et gestionnaires de systèmes de règlement et de livraison d'instruments financiers, les conseillers en investissements financiers, les conseillers en investissements participatifs et les intermédiaires habilités mentionnés à l'article L. 211-4 , ainsi que les placements collectifs mentionnés au I de l'article L. 214-1 et les sociétés de gestion de placements collectifs mentionnées à l'article L. 543-1 et les succursales des sociétés de gestion européennes d'OPCVM et de FIA mentionnées aux articles L. 532-20-1 et L. 532-21-3 ; 6° bis Les prestataires de services d'investissement ayant leur siège social dans un autre Etat membre de l'Union européenne en tant qu'ils exercent leur activité sur le territoire national en ayant recours à des agents liés mentionnés à l' article L. 545-1 du code monétaire et financier ; 7° Les changeurs manuels ; 7° bis Les prestataires des services mentionnés aux 1° et 2° de l'article L. 54-10-2 ; 7° ter Les émetteurs de jetons ayant obtenu le visa mentionné à l'article L. 552-4 dans le cadre de l'offre ayant fait l'objet du visa et dans la limite des transactions avec les souscripteurs prenant part à cette offre ; 7° quater Les prestataires agréés au titre de l'article L. 54-10-5, à l'exception des prestataires mentionnés au 7° bis du présent article ; 8° Les personnes exerçant les activités mentionnées aux 1°, uniquement en ce qui concerne leur activité de location en exécution d'un mandat de transaction de biens immeubles dont le loyer mensuel est supérieur ou égal à 10 000 euros, 2°, 4°, 5° et 8° de l'article 1er de la loi n° 70-9 du 2 janvier 1970 réglementant les conditions d'exercice des activités relatives à certaines opérations portant sur les immeubles et les fonds de commerce ; 9° Les opérateurs de jeux ou de paris autorisés sur le fondement de l'article L. 321-1 et L. 321-3 du code de la sécurité intérieure, du V de l'article 34 de la loi n° 2017-257 du 28 février 2017 relative au statut de Paris et à l'aménagement métropolitain et leurs représentants légaux et directeurs responsables ; 9° bis Les opérateurs de jeux ou de paris autorisés sur le fondement de l'article 21 de la loi n° 2010-476 du 12 mai 2010 relative à l'ouverture à la concurrence et à la régulation du secteur des jeux d'argent et de hasard en ligne et leurs représentants légaux, de l'article 5 de la loi du 2 juin 1891, ayant pour objet de réglementer l'autorisation et le fonctionnement des courses de chevaux, de l'article 137 de la loi n° 2019-486 du 22 mai 2019 relative à la croissance et la transformation des entreprises ; 10° Les personnes qui négocient des œuvres d'art et des antiquités ou agissent en qualité d'intermédiaires dans le commerce des œuvres d'art et d'antiquités, y compris lorsque celui-ci est réalisé par des galeries d'art, lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros et les personnes qui entreposent ou négocient des œuvres d'art ou agissent en qualité d'intermédiaires dans le commerce des œuvres d'art quand celui-ci est réalisé dans des ports francs ou zones franches, lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros ; 11° Les personnes acceptant des paiements en espèces ou au moyen de monnaie électronique d'un montant supérieur à un seuil fixé par décret et se livrant au commerce de biens ; 12° Les experts-comptables, les salariés autorisés à exercer la profession d'expert-comptable en application des articles 83 ter et 83 quater de l'ordonnance n° 45-2138 du 19 septembre 1945 portant institution de l'ordre des experts-comptables et réglementant les titres et la profession d'expert-comptable ; 12° bis Les commissaires aux comptes ; 13° Les avocats au Conseil d'Etat et à la Cour de cassation, les avocats, les notaires, les huissiers de justice, les administrateurs judiciaires, les mandataires judiciaires et les commissaires-priseurs judiciaires, dans les conditions prévues à l'article L. 561-3 ; 14° Les opérateurs de ventes volontaires de meubles aux enchères publiques lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros ; 15° Les personnes exerçant l'activité de domiciliation mentionnée aux articles L. 123-11-2 et suivants du code de commerce ; 16° Les personnes exerçant l'activité d'agents sportifs mentionnés à l' article L. 222-7 du code du sport ; 17° Les personnes autorisées au titre du I de l'article L. 621-18-5 ; 18° Les caisses des règlements pécuniaires des avocats créées en application du 9° de l'article 53 de la loi n° 71-1130 du 31 décembre 1971 au titre des fonds, effets ou valeurs déposés par les avocats pour le compte de leurs clients dans le cadre des activités mentionnées au I de l'article L. 561-3 ; 19° Les greffiers des tribunaux de commerce mentionnés à l'article L. 741-1 du code de commerce. Les personnes assujetties mentionnées aux 1° à 19° comprennent les personnes physiques et les personnes morales."

In [934]:
get_eu_data_3p(document_aml_pdf_step_4, fr_old_version, fr_new_version )

Pour analyser les changements réglementaires dans la loi française et identifier les parties de la directive européenne à l'origine de ces changements, nous devons comparer les versions avant et après l'entrée en vigueur de la directive européenne. Voici les principaux changements et les articles de la directive européenne correspondants :

1. **Ajout des prestataires de services sur crypto-actifs** :
   - **Avant** : Les prestataires de services sur crypto-actifs étaient mentionnés de manière spécifique et détaillée.
   - **Après** : Les prestataires de services sur crypto-actifs sont regroupés sous les articles 7° bis et 7° quater.
   - **Directive européenne** : Article 2(1)(3)(g) et (h) de la directive, qui inclut les prestataires de services sur crypto-actifs dans les entités assujetties.

2. **Modification des seuils pour les transactions en espèces** :
   - **Avant** : Les personnes acceptant des paiements en espèces ou au moyen de monnaie électronique d'un montant supérieur à u

##### 4.2.2 - Exemple 1 : L561-2 du cmf (prompt v2 - 4p)

In [935]:
get_eu_data_4p(document_aml_pdf_step_4, fr_old_version, fr_new_version)

["Remplacer 'paiement,' par 'paiement et'", "Supprimer 'et les prestataires de services sur crypto-actifs'", "Supprimer 'et les prestataires de services sur crypto-actifs'", "Supprimer 'ou d'une ou plusieurs personnes en vue de la fourniture de services sur cryptoactifs'", "Remplacer 'l'article' par 'l' article'", "Remplacer '721-7' par '711-2'", "Remplacer '721-18 et L. 721-19' par '712-4'", "Remplacer '532-48' par '532-48,'", "Remplacer '440-2,' par '440-2 ,'", "Remplacer '421-2,' par '421-2 ,'", "Supprimer 'mentionnés aux 1° et 2° du I de l'article L. 441-1'", "Remplacer 'prestataires de services de financement participatif au titre de leurs activités mentionnées à l'article L. 547-4' par 'conseillers en investissements participatifs'", "Remplacer '211-4,' par '211-4 ,'", "Remplacer 'l'article' par 'l' article'", "Supprimer 'a)'", "Remplacer 'à 4° de l'article L. 54-10-2, et ; b) Les prestataires des services autorisés à fournir des services sur crypto-actifs conformément à l'articl

##### 4.2.2 - Exemple 2 : L561-10 du cmf (prompt v1 - 3p)

In [936]:
fr_old_version="Les personnes mentionnées à l'article L. 561-2 appliquent des mesures de vigilance complémentaires à l'égard de leur client, en sus des mesures prévues aux articles L. 561-5 et L. 561-5-1 , lorsque : 1° Le client ou son représentant légal n'est pas physiquement présent aux fins de l'identification au moment de l'établissement de la relation d'affaires ; 2° Le client, le cas échéant son bénéficiaire effectif, le bénéficiaire d'un contrat d'assurance-vie ou de capitalisation, le cas échéant son bénéficiaire effectif, est une personne qui est exposée à des risques particuliers en raison des fonctions politiques, juridictionnelles ou administratives qu'elle exerce ou a exercées pour le compte d'un Etat ou de celles qu'exercent ou ont exercées des membres directs de sa famille ou des personnes connues pour lui être étroitement associées ou le devient en cours de relation d'affaires ; 3° Le produit ou l'opération présente, par sa nature, un risque particulier de blanchiment de capitaux ou de financement du terrorisme, notamment lorsqu'ils favorisent l'anonymat ; 4° L'opération est une opération pour compte propre ou pour compte de tiers effectuée avec des personnes physiques ou morales, y compris leurs filiales ou établissements, domiciliées, enregistrées ou établies dans un Etat ou un territoire figurant sur les listes publiées par le Groupe d'action financière parmi ceux dont la législation ou les pratiques font obstacle à la lutte contre le blanchiment des capitaux et le financement du terrorisme ou par la Commission européenne en application de l'article 9 de la directive (UE) 2015/849 du 20 mai 2015 relative à la prévention de l'utilisation du système financier aux fins du blanchiment de capitaux ou du financement du terrorisme. S'il n'existe pas de soupçon de blanchiment des capitaux ou de financement du terrorisme, les personnes mentionnées à l'article L. 561-2 peuvent ne pas appliquer aux clients mentionnés aux 1° et 2° les mesures de vigilance complémentaires prévues par le présent article lorsque la relation d'affaires est établie avec une personne mentionnée au 2° de l'article L. 561-9 ou est établie exclusivement pour un ou plusieurs produits mentionnés au même 2° de l'article L. 561-9 . Un décret en Conseil d'Etat précise les catégories de personnes mentionnées au 2°, la liste des produits et des opérations mentionnées au 3°, ainsi que les mesures de vigilance complémentaires."

In [937]:
fr_new_version="Les personnes mentionnées à l'article L. 561-2 appliquent des mesures de vigilance complémentaires à l'égard de leur client, en sus des mesures prévues aux articles L. 561-5 et L. 561-5-1 , lorsque : 1° Le client, le cas échéant son bénéficiaire effectif, le bénéficiaire d'un contrat d'assurance-vie ou de capitalisation, le cas échéant son bénéficiaire effectif, est une personne qui est exposée à des risques particuliers en raison des fonctions politiques, juridictionnelles ou administratives qu'elle exerce ou a exercées ou de celles qu'exercent ou ont exercées des membres directs de sa famille ou des personnes connues pour lui être étroitement associées ou le devient en cours de relation d'affaires ; 2° Le produit ou l'opération présente, par sa nature, un risque particulier de blanchiment de capitaux ou de financement du terrorisme, notamment lorsqu'ils favorisent l'anonymat ; 3° L'opération est une opération pour compte propre ou pour compte de tiers effectuée avec des personnes physiques ou morales, y compris leurs filiales ou établissements ou toute autre entité, domiciliées, enregistrées ou établies dans un Etat ou un territoire figurant sur les listes publiées par le Groupe d'action financière parmi ceux dont la législation ou les pratiques font obstacle à la lutte contre le blanchiment des capitaux et le financement du terrorisme ou par la Commission européenne en application de l'article 9 de la directive (UE) 2015/849 du 20 mai 2015 relative à la prévention de l'utilisation du système financier aux fins du blanchiment de capitaux ou du financement du terrorisme. S'il n'existe pas de soupçon de blanchiment des capitaux ou de financement du terrorisme, les personnes mentionnées à l'article L. 561-2 peuvent ne pas appliquer aux clients mentionnés au 1° les mesures de vigilance complémentaires prévues par le présent article lorsque la relation d'affaires est établie avec une personne mentionnée au 2° de l'article L. 561-9 ou est établie exclusivement pour un ou plusieurs produits mentionnés au même 2° de l'article L. 561-9 . Un décret en Conseil d'Etat précise les catégories de personnes mentionnées au 1°, la liste des produits et des opérations mentionnées au 2°, ainsi que les mesures de vigilance complémentaires."

In [938]:
get_eu_data_3p(document_aml_pdf_step_4, fr_old_version, fr_new_version )

Pour analyser les changements réglementaires dans la loi française et identifier les parties de la directive européenne à l'origine de ces changements, nous devons comparer les versions avant et après de la loi française et les mettre en relation avec les articles pertinents de la directive européenne.

### Analyse des Changements

1. **Suppression de la condition de présence physique pour l'identification (ancien 1°) :**
   - **Avant :** La loi française exigeait des mesures de vigilance complémentaires lorsque le client ou son représentant légal n'était pas physiquement présent pour l'identification.
   - **Après :** Cette condition a été supprimée.

2. **Modification de la définition des personnes exposées à des risques particuliers (ancien 2°, nouveau 1°) :**
   - **Avant :** La loi française incluait les personnes exposées à des risques particuliers en raison de leurs fonctions politiques, juridictionnelles ou administratives, ainsi que les membres directs de leur famille et les p

In [939]:
for chunk in llm_4o.stream(prompt_LCQA_3p):
    print(chunk.content, end="", flush=True)

Pour analyser les changements réglementaires dans la loi française et identifier les parties de la directive européenne à l'origine de ces changements, nous devons examiner les modifications apportées au texte de loi français avant et après l'entrée en vigueur de la directive européenne.

### Analyse des Changements

1. **Ajout de la référence à l'article L. 511-20 du Code monétaire et financier**:
   - **Avant**: Le texte mentionnait des groupes financiers, des sociétés de financement, des groupes au sens de l'article L. 356-1 du code des assurances, des groupes mixtes ou des conglomérats financiers.
   - **Après**: Le texte inclut également des groupes au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2.

### Identification des Articles de la Directive Européenne

Les modifications apportées à la loi française semblent être en réponse aux exigences de la directive européenne concernant la lutte contre le blanchimen

##### 4.2.3 - Exemple 2 : L561-10 du cmf (prompt v2 - 4p)

In [940]:
get_eu_data_4p(document_aml_pdf_step_4, fr_old_version, fr_new_version)

["Supprimer 'client ou son représentant légal n'est pas physiquement présent aux fins de l'identification au moment de l'établissement de la relation d'affaires ; 2° Le'", "Supprimer 'pour le compte d'un Etat'", "Remplacer '3°' par '2°'", "Remplacer '4°' par '3°'", "Remplacer 'établissements,' par 'établissements ou toute autre entité,'", "Remplacer 'aux' par 'au'", "Supprimer 'et 2°'", "Remplacer '2°,' par '1°,'", "Remplacer '3°,' par '2°,'"]


Pour analyser les changements réglementaires dans la loi française et identifier les parties de la directive européenne à l'origine de ces changements, nous allons examiner les modifications apportées à la loi française et les comparer avec les dispositions de la directive européenne.

### Changements dans la loi française

1. **Suppression de la mention du représentant légal non physiquement présent**:
   - **Avant**: "Le client ou son représentant légal n'est pas physiquement présent aux fins de l'identification au moment de l'établissement de la relation d'affaires ; 2° Le"
   - **Après**: Cette mention a été supprimée.

2. **Suppression de la mention "pour le compte d'un État"**:
   - **Avant**: "pour le compte d'un État"
   - **Après**: Cette mention a été supprimée.

3. **Renumérotation des articles**:
   - **Avant**: "3°" et "4°"
   - **Après**: "2°" et "3°"

4. **Ajout de "ou toute autre entité"**:
   - **Avant**: "établissements,"
   - **Après**: "établissements ou toute autre

### 5 - Option 2: Retrieval Augmented Generation

#### 5.1 - Chunking par Article (RecursiveCharacterTextSplitter)

##### 5.1.1 - AML 5

Découper le texte avec le séparateur le plus large (par exemple un double saut de ligne), puis s’il n’arrive pas à produire un chunk qui respecte les contraintes (comme la taille maximale si elle est spécifiée), il descend au séparateur suivant (par exemple un point, puis un espace).

In [941]:
article_splitter = RecursiveCharacterTextSplitter(
    separators=   ["Article"],
    chunk_size= 100, 
# attention spliter	Si le « morceau » résultant dépasse chunk_size, il redescend au séparateur suivant (plus petit).
    chunk_overlap=0)



In [942]:
#chunks = article_splitter.split_text(directive_UE_step_3_trait)
chunks_aml5 = article_splitter.split_text(document_aml_pdf_step_4)
chunks_aml5

["ONT ADOPTÉ LA PRÉSENTE DIRECTIVE: CHAPITRE I DISPOSITIONS GÉNÉRALES SECTION 1 Objet, champ d'application et définitions ",
 "Article premier 1.   La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme. 2.   Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits. 3.   Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les agissements ci-après énumérés, commis intentionnellement:  la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux conséquences juridiques des actes qu'elle a commis;   le fait de dissimuler ou de déguiser la nature, l'origine, l'empl

In [943]:
chunks_aml5_dic = {item[:10]: item[10:].lstrip() for item in chunks_aml5}
chunks_aml5_dic

{'ONT ADOPTÉ': "LA PRÉSENTE DIRECTIVE: CHAPITRE I DISPOSITIONS GÉNÉRALES SECTION 1 Objet, champ d'application et définitions ",
 'Article pr': "emier 1.   La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme. 2.   Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits. 3.   Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les agissements ci-après énumérés, commis intentionnellement:  la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux conséquences juridiques des actes qu'elle a commis;   le fait de dissimuler ou de déguiser la nature, l'origine,

In [944]:
# Affichage des chunks
for idx, chunk in enumerate(chunks_aml5):
    print(f"--- Chunk {idx+1} ---\n{chunk}\n")

--- Chunk 1 ---
ONT ADOPTÉ LA PRÉSENTE DIRECTIVE: CHAPITRE I DISPOSITIONS GÉNÉRALES SECTION 1 Objet, champ d'application et définitions 

--- Chunk 2 ---
Article premier 1.   La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme. 2.   Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits. 3.   Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les agissements ci-après énumérés, commis intentionnellement:  la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux conséquences juridiques des actes qu'elle a commis;   le fait de dissimuler ou de déguiser l

##### 5.1.2 - CRR

In [945]:
directive_UE_CRR_step4

'--- ---   FR L 176/1  * * * RÈGLEMENT (UE) No 575/2013 DU PARLEMENT EUROPÉEN ET DU CONSEIL du 26 juin 2013 concernant les exigences prudentielles applicables aux établissements de crédit et aux entreprises d\'investissement et modifiant le règlement (UE) no 648/2012 (Texte présentant de l\'intérêt pour l\'EEE) LE PARLEMENT EUROPÉEN ET LE CONSEIL DE L\'UNION EUROPÉENNE, vu le traité sur le fonctionnement de l\'Union européenne, et notamment son article 114, vu la proposition de la Commission européenne, après transmission du projet d\'acte législatif aux parlements nationaux, vu l\'avis de la Banque centrale européenne , vu l\'avis du Comité économique et social européen , statuant conformément à la procédure législative ordinaire, considérant ce qui suit:  Dans sa déclaration du 2 avril 2009 sur le renforcement du système financier, le G20 a appelé à agir de manière cohérente au niveau international pour renforcer la transparence, la responsabilité et la réglementation en améliorant l

In [946]:
chunks_crr = article_splitter.split_text(directive_UE_CRR_step4)
chunks_crr

['--- ---   FR L 176/1  * * * RÈGLEMENT (UE) No 575/2013 DU PARLEMENT EUROPÉEN ET DU CONSEIL du 26 juin 2013 concernant les exigences prudentielles applicables aux établissements de crédit et aux entreprises d\'investissement et modifiant le règlement (UE) no 648/2012 (Texte présentant de l\'intérêt pour l\'EEE) LE PARLEMENT EUROPÉEN ET LE CONSEIL DE L\'UNION EUROPÉENNE, vu le traité sur le fonctionnement de l\'Union européenne, et notamment son article 114, vu la proposition de la Commission européenne, après transmission du projet d\'acte législatif aux parlements nationaux, vu l\'avis de la Banque centrale européenne , vu l\'avis du Comité économique et social européen , statuant conformément à la procédure législative ordinaire, considérant ce qui suit:  Dans sa déclaration du 2 avril 2009 sur le renforcement du système financier, le G20 a appelé à agir de manière cohérente au niveau international pour renforcer la transparence, la responsabilité et la réglementation en améliorant 

#### 5.2 - Test of  structure extraction with LLM 

In [947]:
prompt_art_extract = f"""
        You are an expert data scientist and data engineer with an expertise in law .
        That means that you have a deep knowlage of law.
        
        To do: 
          - Extract the structure of this document to prepare to make the best and most reliable embedding.
          
        Rules: 
          - Be as acurate as possible.
          - Do not ommit information.
          - Do not omit the structure under "Article"
          - Do not translate the text : 
          {document_aml_pdf} 
          """

In [948]:
for chunk in llm_4o.stream(prompt_art_extract):
    print(chunk.content, end="", flush=True)

To prepare the best and most reliable embedding for the given document, we need to extract its structure accurately. Here is the detailed structure of the document:

### Metadata
- **Producer**: 3-Heights(TM) PDF Producer 1.91.6.0 (http://www.pdf-tools.com)
- **Creator**: Arbortext Advanced Print Publisher 10.0.1465/W Unicode
- **Creation Date**: 2018-06-15T08:50:29+02:00
- **Author**: Publications Office
- **Modification Date**: 2018-06-15T08:55:10+01:00
- **Title**: Directive (UE) 2018/ du Parlement européen et du Conseil du 30 mai 2018 modifiant la directive (UE) 2015/849 relative à la prévention de l’utilisation du système financier aux fins du blanchiment de capitaux ou du financement du terrorisme ainsi que les directives 2009/138/CE et 2013/36/UE
- **Total Pages**: 32

### Document Content
#### Title
- **DIRECTIVE (UE) 2018/843 DU PARLEMENT EUROPÉEN ET DU CONSEIL du 30 mai 2018 modifiant la directive (UE) 2015/849 relative à la prévention de l’utilisation du système financier au

#### 5.4 - Extract structure with LLM (Whole structure)

#### 5.4.1 - With Gpt 4o

In [949]:
# LLM produit : analyse du changement avant et après et recherche de l article UE a l origine du changement.
prompt_structure_extract_json = f""" 
        You are an expert data scientist and data engineer with an expertise in law .
        That means that you have a deep knowlage of law.
        
        To do: 
          - Extract the whole structure of this document to prepare to make the best and most reliable embedding.
          - The output is a tabular Json. 
          Example de contenu a mettre en accollade : 
           'article 1er', 'paragraphe n°3', 'contenu': "3. Aux fins de la présente directive, sont 
          criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ...".
          
        Rules: 
          - Be as acurate as possible.
          - Do not ommit information.
          - You will be starting from the "Article"
          - Do not omit the structure under "Article" until the  "Article" : Article 69 Les États membres sont destinataires de la présente directive.
          - Do not translate the text : {document_aml_pdf} 
          
"""

In [950]:
for chunk in llm_4o.stream(prompt_structure_extract_json):
    print(chunk.content, end="", flush=True)  # Affichage en temps réel
    output_buffer_4o.write(chunk.content)  # Stockage dans le buffer

# Récup
result_text_extract_4o = output_buffer_4o.getvalue()

```json
[
  {
    "article": "Article premier",
    "paragraphe n°1": {
      "contenu": "Modifications de la directive (UE) 2015/849"
    },
    "paragraphe n°2": {
      "contenu": "La directive (UE) 2015/849 est modifiée comme suit:"
    },
    "paragraphe n°3": {
      "contenu": "1) À l’article 2, paragraphe 1, le point 3) est modifié comme suit:"
    },
    "paragraphe n°4": {
      "contenu": "a) le point a) est remplacé par le texte suivant:"
    },
    "paragraphe n°5": {
      "contenu": "«a) les auditeurs, experts-comptables externes et conseillers fiscaux, et toute autre personne qui s’engage à fournir, directement ou par le truchement d’autres personnes auxquelles cette autre personne est liée, une aide matérielle, une assistance ou des conseils en matière fiscale comme activité économique ou professionnelle principale;»;"
    },
    "paragraphe n°6": {
      "contenu": "b) le point d) est remplacé par le texte suivant:"
    },
    "paragraphe n°7": {
      "contenu": "«d)

In [951]:
count_tokens(result_text_extract_4o)

4096

- Problem of reaching max token 4K for GPT 4o.
- Let's use 4o mini for higher context windows, lower cost for this low complicated tasked that do not require a deep undersating of the content.

#### 5.4.2 - With Gpt 4o mini

In [952]:
output_buffer_4o_mini = io.StringIO()

In [953]:
for chunk in llm_4omini.stream(prompt_structure_extract_json):
    print(chunk.content, end="", flush=True)  # Affichage en temps réel
    output_buffer_4o_mini.write(chunk.content)  # Stockage dans le buffer

```json
{
  "articles": [
    {
      "article": "Article 1er",
      "paragraphes": [
        {
          "paragraphe": "1",
          "contenu": "La directive (UE) 2015/849 est modifiée comme suit: 1) À l’article 2, paragraphe 1, le point 3) est modifié comme suit: a) le point a) est remplacé par le texte suivant: «a) les auditeurs, experts-comptables externes et conseillers fiscaux, et toute autre personne qui s’engage à fournir, directement ou par le truchement d’autres personnes auxquelles cette autre personne est liée, une aide matérielle, une assistance ou des conseils en matière fiscale comme activité économique ou professionnelle;»;"
        },
        {
          "paragraphe": "2",
          "contenu": "b) le point d) est remplacé par le texte suivant: «d) les agents immobiliers, y compris lorsqu’ils agissent en qualité d’intermédiaires pour la location de biens immeubles, mais uniquement en ce qui concerne les transactions pour lesquelles le loyer mensuel est d’un montant ég

In [954]:
# Récup
result_text_extract_4o_mini = output_buffer_4o_mini.getvalue()

In [955]:
count_tokens(result_text_extract_4o_mini)

829

#### 5.4.4 - Embedding

In [956]:
from langchain.vectorstores import Chroma

In [957]:
sentence1 = "test test test "
emb1 = embeddings.embed_query(sentence1)
print(emb1)

[-0.015937637261025525, 0.01671048597292706, -0.006908508829043585, 0.04772809064974347, 0.024731142017042972, 0.010593675824367968, -0.01671048597292706, 0.0607345615214083, -0.02684233849833496, 0.015145940443186146, -0.0013006470173172909, 0.032893172825531114, -0.021507801114646327, -0.007940544608960876, 0.004281296320438067, 0.026012939880684538, -0.005183738100686199, 0.018180783522398266, -0.033873371191845254, -0.002001629641919673, 0.0037016602521731975, -0.01896305535594616, -0.007082871038436004, 0.01292164508171893, -0.037699911088896754, 0.016079012025397757, 0.027841384970586948, -0.00581521158255422, 0.0010550084806358584, -0.03172447849867757, 0.007115858483456191, 0.03445772021791715, -0.0029547307127345207, 0.004639445723514387, 0.018774557532761638, 0.0009672383726290914, 0.01829388333389605, 0.024259894665113985, 0.007605957200951982, 0.0026319252335818984, -0.004677145660680316, -0.022733047209893883, -0.0151176654903117, -0.00016508438861708629, -0.02203559837232

#### 5.4.5 - Vector store with Chroma

In [959]:
vectordb = Chroma.from_texts( 
    texts=chunks_aml5,
    metadatas= [{"source": "AML_5"} for _ in chunks_aml5],
    persist_directory="/Users/oussa/Desktop/Github_perso/Chroma_DB_All",
    embedding=embeddings)

In [961]:
vectordb = Chroma.from_texts( 
    texts=chunks_crr,
    metadatas= [{"source": "CRR"} for _ in chunks_crr],
    persist_directory="/Users/oussa/Desktop/Github_perso/Chroma_DB_All",
    embedding=embeddings)

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}

In [962]:
print(vectordb._collection.count()) #same as nbr of chunks  we have
#vectordb.delete_collection()

70


In [1056]:
question = "Est est l'objectif de la directive ?"
chunks_aml5[1]

"Article premier 1.   La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme. 2.   Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits. 3.   Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les agissements ci-après énumérés, commis intentionnellement:  la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux conséquences juridiques des actes qu'elle a commis;   le fait de dissimuler ou de déguiser la nature, l'origine, l'emplacement, la disposition, le mouvement ou la propriété réels de biens ou des droits qui y sont liés, dont celui qui s'y livre s

In [964]:
docs = vectordb.similarity_search(question,k=4, filter={"source":"AML_5"})
docs

[Document(metadata={'source': 'AML_5'}, page_content="ONT ADOPTÉ LA PRÉSENTE DIRECTIVE: CHAPITRE I DISPOSITIONS GÉNÉRALES SECTION 1 Objet, champ d'application et définitions "),
 Document(metadata={'source': 'AML_5'}, page_content="Article 67 1.   Les États membres mettent en vigueur les dispositions législatives, réglementaires et administratives nécessaires pour se conformer à la présente directive au plus tard le 26 juin 2017. Ils communiquent immédiatement à la Commission le texte de ces dispositions. Lorsque les États membres adoptent ces dispositions, celles-ci contiennent une référence à la présente directive ou sont accompagnées d'une telle référence lors de leur publication officielle. Les modalités de cette référence sont arrêtées par les États membres. 2.   Les États membres communiquent à la Commission le texte des dispositions essentielles de droit interne qu'ils adoptent dans le domaine régi par la présente directive. "),
 Document(metadata={'source': 'AML_5'}, page_conte

##### 5.4.5.2 - max_marginal_relevance_search - Only many documents

In [968]:
vectordb.max_marginal_relevance_search(question,k=4,
                                      fetch_k=3) # here diversity factor
docs_ss = vectordb.similarity_search(question,k=4)
docs_ss

[Document(metadata={'source': 'AML_5'}, page_content="ONT ADOPTÉ LA PRÉSENTE DIRECTIVE: CHAPITRE I DISPOSITIONS GÉNÉRALES SECTION 1 Objet, champ d'application et définitions "),
 Document(metadata={'source': 'AML_5'}, page_content="Article 67 1.   Les États membres mettent en vigueur les dispositions législatives, réglementaires et administratives nécessaires pour se conformer à la présente directive au plus tard le 26 juin 2017. Ils communiquent immédiatement à la Commission le texte de ces dispositions. Lorsque les États membres adoptent ces dispositions, celles-ci contiennent une référence à la présente directive ou sont accompagnées d'une telle référence lors de leur publication officielle. Les modalités de cette référence sont arrêtées par les États membres. 2.   Les États membres communiquent à la Commission le texte des dispositions essentielles de droit interne qu'ils adoptent dans le domaine régi par la présente directive. "),
 Document(metadata={'source': 'AML_5'}, page_conte

##### 5.4.5.3 - ContextualCompressionRetriever (for long articles ?)


Contextual compression is meant to fix document with a lot of irrelevant text that can lead to more expensive LLM calls and poorer responses.

##### 5.4.6 - Q&A - with RetrievalQA

##### old version: 

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI

# Initialiser le modèle de langage
llm = OpenAI()

# Définir le prompt
system_prompt = (
    "Utilisez le contexte donné pour répondre à la question. "
    "Si vous ne connaissez pas la réponse, dites que vous ne savez pas. "
    "Utilisez trois phrases maximum et soyez concis. "
    "Contexte : {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Créer la chaîne de réponse aux questions
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Créer la chaîne de récupération
chain = create_retrieval_chain(retriever, question_answer_chain)

# Exécuter la chaîne avec votre question
resultat = chain.invoke({"input": Question2})

In [969]:
from langchain.chains import RetrievalQA

In [970]:
qa_chain = RetrievalQA.from_chain_type(
    llm_4o,
    retriever=vectordb.as_retriever()
)

In [974]:
Question2= "quels articles élargissent le champ d'application des entités assujetties aux obligations de lutte contre le blanchiment de capitaux et le financement du terrorisme?"

In [1017]:
resultat = qa_chain({"query" :Question2})

In [1018]:
resultat

{'query': "quels articles élargissent le champ d'application des entités assujetties aux obligations de lutte contre le blanchiment de capitaux et le financement du terrorisme?",
 'result': "Les articles qui élargissent le champ d'application des entités assujetties aux obligations de lutte contre le blanchiment de capitaux et le financement du terrorisme sont les articles 2 et 4.\n\n- L'article 2 énumère les entités assujetties, telles que les établissements de crédit, les établissements financiers, les auditeurs, les experts-comptables externes, les conseillers fiscaux, les notaires, les agents immobiliers, et d'autres personnes physiques ou morales agissant dans l'exercice de leur activité professionnelle.\n\n- L'article 4 permet aux États membres d'étendre le champ d'application de la directive à des professions et à des catégories d'entreprises autres que celles visées à l'article 2, paragraphe 1, qui exercent des activités particulièrement susceptibles d'être utilisées à des fins

##### new version: 

In [1049]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate


In [1060]:
# Définir le prompt
system_prompt = (
    "Utilisez le contexte donné pour répondre à la question. "
    "Si vous ne connaissez pas la réponse, dites que vous ne savez pas. "
    "Utilisez trois phrases maximum et soyez concis. "
    "Contexte : {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Créer la chaîne de réponse aux questions
question_answer_chain = create_stuff_documents_chain(llm_4omini, prompt)

retriever = vectordb.as_retriever(search_kwargs={"k": 4})

# Créer la chaîne de récupération
chain = create_retrieval_chain(retriever, question_answer_chain)

# Exécuter la chaîne avec votre question
resultat = chain.invoke({"input": question})
resultat

{'input': "Est est l'objectif de la directive ?",
 'context': [Document(metadata={'source': 'AML_5'}, page_content="ONT ADOPTÉ LA PRÉSENTE DIRECTIVE: CHAPITRE I DISPOSITIONS GÉNÉRALES SECTION 1 Objet, champ d'application et définitions "),
  Document(metadata={'source': 'AML_5'}, page_content="Article 67 1.   Les États membres mettent en vigueur les dispositions législatives, réglementaires et administratives nécessaires pour se conformer à la présente directive au plus tard le 26 juin 2017. Ils communiquent immédiatement à la Commission le texte de ces dispositions. Lorsque les États membres adoptent ces dispositions, celles-ci contiennent une référence à la présente directive ou sont accompagnées d'une telle référence lors de leur publication officielle. Les modalités de cette référence sont arrêtées par les États membres. 2.   Les États membres communiquent à la Commission le texte des dispositions essentielles de droit interne qu'ils adoptent dans le domaine régi par la présente di

##### 5.4.7 - Q&A map rerank

##### old 

Si dépassement de la fenetre contextuelle: 

- map_reduce
- refine : successivement chaque document suivant pour affiner, corriger ou enrichir cette réponse à chaque étape. lorsque chaque document apporte un complément d’information à la réponse
- map_rerank: Quand on veut identifier la meilleure réponse unique parmi plusieurs candidats

In [1038]:
from langchain.chains import MapRerankDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.output_parsers import RegexParser
from langchain.docstore.document import Document

# Définir le modèle de prompt
prompt_template = (
    "Utilisez le contexte suivant pour répondre à la question posée. "
    "Fournissez votre réponse suivie d'un score de confiance entre 0 et 100.\n\n"
    "Contexte : {context}\n"
    "Question : {question}\n"
    "Réponse :"
)

# Configurer le parser de sortie
output_parser = RegexParser(
    regex=r"(.*?)\nScore: (\d+)",
    output_keys=["answer", "score"],
)

# Créer le PromptTemplate
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser,
)

# Créer la chaîne LLM
llm_chain = LLMChain(llm=llm_4omini, prompt=prompt)

# Créer la chaîne de reclassement
map_rerank_chain = MapRerankDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    rank_key="score",
    answer_key="answer"
)

# Exemple de documents
documents = [
    Document(page_content="LangChain est une bibliothèque pour le développement d'applications basées sur des modèles de langage."),
    Document(page_content="LangChain offre des outils pour la gestion des invites et l'intégration de modèles."),
    Document(page_content="LangChain facilite la récupération de documents et bien plus encore.")
]

# Poser une question et obtenir une réponse
question = "Qu'est-ce que LangChain ?"
result = map_rerank_chain.combine_docs(documents, question=question)

# Afficher la réponse sélectionnée
print(result)

/Users/oussa/Desktop/Github_perso/Advanced_RAG/venv/lib/python3.13/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


ValueError: Could not parse output: LangChain est une bibliothèque conçue pour le développement d'applications qui s'appuient sur des modèles de langage. Elle facilite l'intégration et l'utilisation de ces modèles dans divers contextes d'application. 

Score de confiance : 90

In [1070]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors.rankllm_rerank import RankLLMRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever



In [1073]:


# Ajouter le re-ranking à la récupération
document_compressor = RankLLMRerank(llm=llm_4omini, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=document_compressor,
    base_retriever=retriever
)

# Créer la chaîne complète avec récupération et re-ranking
chain_with_rerank = create_retrieval_chain(compression_retriever, question_answer_chain)

# Exécuter la chaîne avec votre question
resultat = chain_with_rerank.invoke({"input": Question2})


PackageNotFoundError: No package metadata was found for rank_llm

#### 5.4.8 BM 25

In [1043]:
from langchain_community.retrievers import BM25Retriever

In [1044]:
chunks = [
"Python was created by Guido van Rossum and released in 1991.",
"Python is a popular language for machine learning (ML).",
"The PyTorch library is a popular Python library for AI and ML."]


In [ ]:
bm25_retriever = BM25Retriever.from_texts(chunks, k=3)

In [1047]:
results = bm25_retriever.invoke("When was Python created?")
print("Most Relevant Document:")
print(results[0].page_content)

Most Relevant Document:
Python was created by Guido van Rossum and released in 1991.
